##### OpenAI 인증키 설정

In [1]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

Fy


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print("===>1. 문서 로딩")
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()

===>1. 문서 로딩


In [4]:
print("==> 2. 문서 분할 → 작은 청크로 나누기")
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = pythontext_splitter.split_documents(documents)

==> 2. 문서 분할 → 작은 청크로 나누기


In [5]:
print("==> 3. 벡터화 → 임베딩으로 변환")
pythonembeddings = UpstageEmbeddings(model="solar-embedding-1-large")

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, pythonembeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")
# 로컬 파일로 저장
vectorstore.save_local("faiss_db")

pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)
print(" Retriever 설정 완료")

==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (104개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료


In [6]:
print("===> 6. 생성 → LLM으로 답변 생성")
pythonllm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.2
    )

pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""
prompt = PromptTemplate(
    template=pythonprompt_template,
    input_variables=["context", "question"]
)

print(" 프롬프트 설정 완료")
print("\n ===> 7.  QA 체인 생성...")
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)
print("  RAG 파이프라인 구축 완료!")

pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)

===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 RAG 시스템 테스트

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 소멸 시 복구 요구 대응 방안  

#### 1. **사례 기반 분석**  
제시된 사례들을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 결과가 달라집니다:  
- **(1) 계정 소유권 확인**:  
  - 2006년 사례(1-가-1)에서는 신청인이 계정 명의자가 아니라는 이유로 복구가 거절되었습니다. 게임사의 약관에서 **계정 공유 및 현금거래를 금지**하는 경우, 실소유주라도 법적 보호를 받기 어렵습니다.  
  - **해결 방안**: 계정 명의자 본인임을 증명할 수 있는 자료(구매 영수증, 결제 내역 등)를 제출해야 합니다.  

- **(2) 시스템 오류 입증 가능성**:  
  - 2009년 사례(1,000억 게임머니 소실)에서는 **다른 이용자의 유사 피해 신고가 없어** 시스템 오류로 인정되지 않았습니다.  
  - 2006년 프로그램 오류 사례(4번)에서는 게임사가 오류를 인정하고 복구한 반면, 2009년 사례(23번)에서는 **통신 환경 문제로 판단**되어 거절되었습니다.  
  - **해결 방안**: 오류 발생 시점의 **로그 기록, 스크린샷, 게임사 공지** 등을 확보하여 시스템 오류를 입증해야 합니다.  

- **(3) 게임사의 약관 및 정책**:  
  - 2007년 사례(5번)에서는 **해킹 아이템 회수**가 약관에 따라 정당하다고 판단되었습니다.  
  - **해결 방안**: 게임사의 **이용약관**에서 "시스